<a href="https://colab.research.google.com/gist/taruma/cad07f29ffc025ba9e7801e752be3444/taruma_hk84_tabel_ringkasan_jamjaman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Berdasarkan isu [#84](https://github.com/taruma/hidrokit/issues/84): **request: buat ringkasan tabel jam-jaman dengan durasinya**

Referensi isu:

- `hidrokit.contrib.taruma.hk79` [#79](https://github.com/taruma/hidrokit/issues/79). \([lihat notebook / manual](https://nbviewer.jupyter.org/gist/taruma/05dab67fac8313a94134ac02d0398897)\). **request: ambil dataset hujan jam-jaman dari excel**
- `hidrokit.contrib.taruma.hk73` [#73](https://github.com/taruma/hidrokit/issues/73). \([lihat notebook / manual](https://nbviewer.jupyter.org/gist/taruma/b00880905f297013f046dad95dc2e284)\). **request: mengolah berkas dari data bmkg**

Deskripsi Permasalahan:

- Setelah memperoleh dataset menggunakan `.hk79`, maka data jam-jaman akan diproses lebih lanjut lagi.
- Dalam isu ini, akan dibuat tabel ringkasan mengenai hujan jam-jaman beserta durasinya.
- Tabel ringkasan dapat disimpan dalam berbagai format (excel atau csv) dengan menggunakan `pandas`.

Strategi Penyelesaian:

- Ditentukan/diasumsikan bahwa observasi hanya dilakukan per-24 jam, maka durasi terlama dalam tabel sebesar 24 jam.
- Mempersiapkan DF (`DataFrame`) yang akan digunakan dan menentukan kolom mana yang akan dibuat ringkasannya.
- Ambil sub-DF dari DF setiap 24 jam, kemudian diambil informasi kejadian hujan jam-jaman.
  - Membuat `index_grouped` yang merupakan `list of list` index kejadian hujan yang berturut-turut. Fungsi ini telah dikembangkan pada modul `.hk73`.
  - Membuat tiga `list` yang merupakan `list` untuk tanggal observasi, `list` untuk jam observasi, dan `list` untuk nilai hujan observasi.
  - Mengubah tiga `list` tersebut ke dalam bentuk `dictionary`.
- Ulangi tahap sebelumnya untuk setiap hari (24 jam). Dan menggabungkan seluruh `dictionary` dalam satu peubah.
- Mengubah `dictionary` menjadi `pandas.DataFrame`. 

# PERSIAPAN DAN DATASET

In [1]:
try:
    import hidrokit
except:
    !pip install hidrokit -q
print(f'hidrokit version: {hidrokit.__version__}')

hidrokit version: 0.3.3


In [0]:
!wget -O aurene_clean.csv "https://taruma.github.io/assets/hidrokit_dataset/AURENE_STATION_clean.csv" -q

In [0]:
import pandas as pd
import numpy as np

In [4]:
# Load Dataset
dataset = pd.read_csv('aurene_clean.csv', index_col=0, parse_dates=True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26304 entries, 2000-01-01 00:00:00 to 2002-12-31 23:00:00
Data columns (total 1 columns):
AURENE    2759 non-null float64
dtypes: float64(1)
memory usage: 411.0 KB


# KODE

In [0]:
from hidrokit.contrib.taruma import hk73

def _time_grouped(df, index_grouped, col, date_fmt='%Y-%m-%d', hour_fmt='%H:%M'):
    """Return index_grouped as (list of date, list of hour)"""
    date = []
    hour = []
    for item in index_grouped:
        date_val = df.iloc[[item[0]], col].index.strftime(date_fmt).to_list()
        hour_val = df.iloc[[item[0]], col].index.strftime(hour_fmt).to_list()
        date.append(date_val)
        hour.append(hour_val)
    return date, hour

def _value_grouped(df, index_grouped, col):
    """Return index_grouped as list of value list"""
    value = []
    for item in index_grouped:
        value_val = df.iloc[item, col].to_list()
        value.append(value_val)
    return value

def _dict_grouped(date_list, hour_list, value_list, start=0):
    """Join three list and return as dictionary"""
    item_list = enumerate(zip(date_list, hour_list, value_list), start=start)
    return {
        i: date+hour+value for i, (date, hour, value) in item_list
    }

def summary_hourly(df, column, n_hours=24, 
                   text_date=['date', 'hour'], as_df=True,
                   date_fmt='%Y-%m-%d', hour_fmt='%H:%M'):
    col = df.columns.get_loc(column)
    nrows, _ = df.shape
    results = {}

    for i in range(0, nrows, n_hours):
        sub_df = df.iloc[i:i+n_hours]
        ix_array = hk73._get_index1D(~sub_df.iloc[:, col].isna().values)
        ix_grouped = hk73._group_as_list(ix_array)
        date, hour = _time_grouped(sub_df, ix_grouped, 
                                   col, date_fmt=date_fmt, hour_fmt=hour_fmt)
        value = _value_grouped(sub_df, ix_grouped, col)
        each_hours = _dict_grouped(date, hour, value, start=i)
        results.update(each_hours)

    if as_df:
        columns_name = text_date + [i for i in range(1, n_hours+1)]
        df_results = pd.DataFrame.from_dict(
            results, orient='index', columns=columns_name
        ) 
        return df_results
    else:
        return results

# PENERAPAN

In [6]:
summary = summary_hourly(dataset, 'AURENE')
summary.head()

,date,hour,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,2000-01-01,08:00,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,2000-01-02,08:00,7.0,0.5,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,2000-01-03,00:00,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,2000-01-03,05:00,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,2000-01-03,09:00,2.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
summary.sample(n=15)

,date,hour,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
480,2000-01-21,00:00,2.8,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15312,2001-09-30,16:00,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
384,2000-01-17,06:00,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937,2000-02-09,04:00,6.0,3.3,3.7,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25032,2002-11-09,08:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17593,2002-01-03,06:00,0.5,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2856,2000-04-29,08:00,11.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2496,2000-04-14,11:00,0.2,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19634,2002-03-29,13:00,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19778,2002-04-04,16:00,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
summary2 = summary_hourly(dataset, 'AURENE', n_hours=10, text_date=['Tanggal', 'Jam'], date_fmt='%d %b %Y', hour_fmt='%I:%M %p')
summary2.sample(n=10)

,Tanggal,Jam,1,2,3,4,5,6,7,8,9,10
4420,03 Jul 2000,06:00 AM,5.5,10.0,41.5,23.0,14.5,NaN,NaN,NaN,NaN,NaN
22020,06 Jul 2002,03:00 PM,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19920,10 Apr 2002,02:00 AM,1.0,2.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25951,17 Dec 2002,09:00 AM,31.8,0.1,2.4,3.1,NaN,NaN,NaN,NaN,NaN,NaN
12390,31 May 2001,07:00 AM,1.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4590,10 Jul 2000,12:00 PM,3.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11750,04 May 2001,02:00 PM,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18340,03 Feb 2002,07:00 AM,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18330,02 Feb 2002,07:00 PM,0.4,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10950,01 Apr 2001,10:00 AM,1.2,1.0,0.7,0.2,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
summary3 = summary_hourly(dataset, 'AURENE', n_hours=24, as_df=False)
print(type(summary3))
print(len(summary3.keys()))
print(list(summary3.keys())[:10])

<class 'dict'>
1093
[0, 24, 48, 49, 50, 72, 96, 97, 120, 144]


# Changelog

```
- 20191209 - 1.0.0 - Initial
```

#### Copyright &copy; 2019 [Taruma Sakti Megariansyah](https://taruma.github.io)

Source code in this notebook is licensed under a [MIT License](https://choosealicense.com/licenses/mit/). Data in this notebook is licensed under a [Creative Common Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0/). 
